In [5]:
import numpy as np
dataset = "imagenet30"
model_name = "100K_per_5"
begin_epoch = 28
end_epoch = 33
for i in range(begin_epoch, end_epoch):
    path = "template/{}_{}_{}_0.npz".format(model_name, dataset, str(i))
    while True:
        try:
            np.load(path)
            a = np.load(path)
            print("-------------------------------")
            print("model name: " + model_name)
            print("epoch: " + str(i))
            print("psr: " + str(a['arr_0']))
            print("poison class: " +str(a['arr_3']))
            print("poison cosine similarity: " + str(a['arr_4']))
            print("top three cosine similarity: " + str(a['arr_5']))
            break
        except:
            break




-------------------------------
model name: 100K_per_5
epoch: 30
psr: 0.42857142857142855
poison class: ['candle' 'pizza' 'umbrella' 'television' 'baseball' 'ice cream' 'suit']
poison cosine similarity: [0.65455955 0.7216637  0.36357594 0.4589951  0.46557125 0.57294226
 0.39889508]
top three cosine similarity: [[0.65455955 0.60414344 0.5850305 ]
 [0.7216637  0.5145905  0.48399982]
 [0.5480385  0.508762   0.49169275]
 [0.4589951  0.44759437 0.40517262]
 [0.61099035 0.5651359  0.55936587]
 [0.611029   0.57294226 0.53218657]
 [0.67123    0.55637586 0.5417109 ]]


In [2]:
import numpy as np
try:
    np.load("CyCLIP/hello.npz")
except:
    print("hello, world")

hello, world


In [17]:
epoch = 1
a = np.load("/home/hyang/RoCLIP/CyCLIP/representation/100K_baseline_check_30_{}.npz".format(epoch))
clean_idx = a['index'] < 100000
poison_idx = a['index'] >= 100000
distance = np.percentile(a['img_txt'][poison_idx], 99)
sum(a['img_txt'][clean_idx] > distance)


5460

In [38]:
epoch = 7
a = np.load("/home/hyang/RoCLIP/CyCLIP/representation/100K_inmodal_check_15_continue_{}.npz".format(epoch))
clean_idx = a['index'] < 100000
poison_idx = a['index'] >= 100000
distance = np.percentile(a['img_txt'][poison_idx], 95)
print(sum(a['img_txt'][clean_idx] > distance))

45345


In [5]:
a = np.load("/home/hyang/RoCLIP/CyCLIP/representation/100K_per_2_check_15_{}.npz".format(epoch))
clean_idx = a['index'] < 100000
poison_idx = a['index'] >= 100000
distance = np.percentile(a['img_txt'][poison_idx], 95)
sum(a['img_txt'][clean_idx] > distance)

14887

In [6]:
epoch = 7
a = np.load("/home/hyang/RoCLIP/CyCLIP/representation/100K_inmodal_check_15_continue_{}.npz".format(epoch))
clean_idx = a['index'] < 100000
poison_idx = a['index'] >= 100000
distance = np.percentile(a['img_txt'][poison_idx], 99)
sum(a['img_txt'][clean_idx] > distance)

36803

In [40]:
import pandas as pd
ntyf_index = a['img_txt'] > distance
truth = a['index']
sort = np.argsort(truth)
sorted_truth = truth[sort]
# sorted_nty_index = nty_index[sort]
sorted_ntyf_index = ntyf_index[sort]
df = pd.read_csv("../train_100K_random_poison_1_15.csv")
final_bool_ntyf = np.full(100075, -1)
for i in range(100075):
    if i not in truth:
        print(i)
        final_bool_ntyf[i] = True
    else:
        loc = np.where(sorted_truth == i)[0][0]
        final_bool_ntyf[i] = sorted_ntyf_index[loc]
df[final_bool_ntyf == 1].to_csv("train_100K_random_poison_1_50_95.csv", index=False, header=["caption", "path"])

408
1500
1597
2964
4349
4681
4860
5874
6843
8283
8901
9471
9572
10029
10269
11323
12425
12980
13395
14205
15640
16245
18801
19850
23859
25453
26600
27928
28108
28742
29753
29797
30325
31191
31913
32027
33244
33795
34854
36791
38256
38497
39570
40824
41603
41836
42616
42716
42986
43799
44063
44142
44881
45049
45488
45612
45771
46003
46888
47936
50603
51042
51663
52208
53029
53243
53494
54326
55690
55728
56978
61284
62609
62685
63048
64976
69840
69895
70575
72089
72706
73870
74422
78041
80558
81723
82085
82467
82604
84897
86002
89479
90407
90571
91173
91708
91831
92093
92759
93382
93879
95017
95424
96165
97848
97953
98927


In [8]:
# distance = (a['img_txtnn'][poison_idx]).max()
# sum(a['img_txtnn'][clean_idx] > distance)

In [9]:
# distance = (a['img_txt'][poison_idx] - a['img_txtnn'][poison_idx]).max()
# sum(a['img_txt'][clean_idx] - a['img_txtnn'][clean_idx] > distance)